In [33]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer

In [34]:
def preprocessing(data_path):    
    raw_data = [[], []]

    with open(data_path) as file:
        lines = file.readlines()

        for line in lines:
            data_x, data_y = line.split(" ")
            raw_data[0].append(data_x)
            raw_data[1].append(data_y[:-1])

    raw_data = np.array(raw_data)
    # print(raw_data.shape)

    data_x = raw_data[0]
    data_y = raw_data[1]
    data = np.append(data_x, data_y)
    # print(len(data))

    input_characters = []
    output_characters = []

    for x in data_x:
        for char in x:
            if char not in input_characters and char not in [" "]:
                input_characters.append(char)

    for y in data_y:
        for char in y:
            if char not in output_characters and char not in [" "]:
                output_characters.append(char)

    input_characters = sorted(input_characters)
    output_characters = sorted(output_characters)

    input_data = np.zeros((len(data_x), max([len(txt) for txt in data_x]), len(input_characters)), dtype = "float32")
    decoder_input_data = np.zeros((len(data_y), max([len(txt) for txt in data_y]), len(output_characters)), dtype = "float32")
    output_data = np.zeros((len(data_y), max([len(txt) for txt in data_y]), len(output_characters)), dtype = "float32")

    input_char_ind = dict([(char, i) for i, char in enumerate(input_characters)])
    output_char_ind = dict([(char, i) for i, char in enumerate(output_characters)])

    for i, (input_text, target_text) in enumerate(zip(data_x, data_y)):
        for t, char in enumerate(input_text):
            input_data[i, t, input_char_ind[char]] = 1.0
        for t, char in enumerate(target_text):
            decoder_input_data[i, t, output_char_ind[char]] = 1.0
            if t > 0:
                output_data[i, t - 1, output_char_ind[char]] = 1.0

    train_length = int(len(input_data)*0.8)

    train_input_data = input_data[:train_length]
    train_decoder_input_data = decoder_input_data[:train_length]
    train_output_data = output_data[:train_length]

    test_input_data = input_data[train_length:]
    test_decoder_input_data = decoder_input_data[train_length:]
    test_output_data = output_data[train_length:]

    return train_input_data, train_decoder_input_data, train_output_data, test_input_data, test_decoder_input_data, test_output_data, input_characters, output_characters




In [35]:
train_input_data, train_decoder_input_data, train_output_data, test_input_data, test_decoder_input_data, test_output_data, input_characters, output_characters = preprocessing("data.txt")

In [36]:
def lstm2_model(input_characters, output_characters):
    # Define an input sequence and process it.
    encoder_inputs = keras.Input(shape=(None, len(input_characters)))
    encoder = keras.layers.LSTM(128, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)

    # We discard `encoder_outputs` and only keep the states.
    encoder_states = [state_h, state_c]

    # Set up the decoder, using `encoder_states` as initial state.
    decoder_inputs = keras.Input(shape=(None, len(output_characters)))

    # We set up our decoder to return full output sequences,
    # and to return internal states as well. We don't use the
    # return states in the training model, but we will use them in inference.
    decoder_lstm = keras.layers.LSTM(128, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = keras.layers.Dense(len(output_characters), activation="softmax")
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model that will turn
    # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

    model.compile(
        optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
    )

    return model

In [37]:
def lstm_model(input_characters, output_characters):
    # Define an input sequence and process it.
    encoder_inputs = keras.Input(shape=(None, len(input_characters)))
    encoder = keras.layers.SimpleRNN(128, return_state=True, activation = 'tanh')
    encoder_outputs, state = encoder(encoder_inputs)

    encoder_states = [state]

    decoder_inputs = keras.Input(shape=(None, len(output_characters)))

    decoder_lstm = keras.layers.SimpleRNN(128, return_sequences=True, return_state=True, activation = 'tanh')
    decoder_outputs, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = keras.layers.Dense(len(output_characters), activation="softmax")
    decoder_outputs = decoder_dense(decoder_outputs)

    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

    return model

In [38]:
def train_model(model, train_input_data, train_decoder_input_data, train_output_data):       
    model.fit(
        [train_input_data, train_decoder_input_data],
        train_output_data,
        batch_size=64,
        epochs=20,
        validation_split=0.2,
    )

In [39]:
def evaluate_model(model, test_input_data, test_decoder_input_data, test_output_data):
    encoder_loss, decoder_loss = model.evaluate([test_input_data, test_decoder_input_data], test_output_data)

    return encoder_loss, decoder_loss

In [40]:
model = lstm_model(input_characters, output_characters)
train_model(model, train_input_data, train_decoder_input_data, train_output_data)
encoder1_loss, decoder1_loss = evaluate_model(model, test_input_data, test_decoder_input_data, test_output_data)

Epoch 1/20
66/66 [==============================] - 3s 30ms/step - loss: 1.2458 - accuracy: 0.0725 - val_loss: 1.2210 - val_accuracy: 0.0612
Epoch 2/20
66/66 [==============================] - 2s 26ms/step - loss: 1.2192 - accuracy: 0.0732 - val_loss: 1.2283 - val_accuracy: 0.0780
Epoch 3/20
66/66 [==============================] - 2s 26ms/step - loss: 1.2336 - accuracy: 0.0768 - val_loss: 1.2418 - val_accuracy: 0.0745
Epoch 4/20
66/66 [==============================] - 2s 27ms/step - loss: 1.2415 - accuracy: 0.0678 - val_loss: 1.2436 - val_accuracy: 0.0625
Epoch 5/20
66/66 [==============================] - 2s 26ms/step - loss: 1.2425 - accuracy: 0.0627 - val_loss: 1.2458 - val_accuracy: 0.0633
Epoch 6/20
66/66 [==============================] - 2s 27ms/step - loss: 1.2430 - accuracy: 0.0622 - val_loss: 1.2447 - val_accuracy: 0.0622
Epoch 7/20
66/66 [==============================] - 2s 26ms/step - loss: 1.2423 - accuracy: 0.0620 - val_loss: 1.2436 - val_accuracy: 0.0623
Epoch 8/20
66

In [41]:
model2 = lstm2_model(input_characters, output_characters)
train_model(model2, train_input_data, train_decoder_input_data, train_output_data)
encoder2_loss, decoder2_loss = evaluate_model(model2, test_input_data, test_decoder_input_data, test_output_data)

Epoch 1/20
66/66 [==============================] - 10s 100ms/step - loss: 1.1953 - accuracy: 0.0774 - val_loss: 1.1550 - val_accuracy: 0.0845
Epoch 2/20
66/66 [==============================] - 6s 87ms/step - loss: 1.1490 - accuracy: 0.0839 - val_loss: 1.2163 - val_accuracy: 0.0713
Epoch 3/20
66/66 [==============================] - 6s 88ms/step - loss: 1.1305 - accuracy: 0.0896 - val_loss: 1.1503 - val_accuracy: 0.0840
Epoch 4/20
66/66 [==============================] - 6s 88ms/step - loss: 1.1063 - accuracy: 0.0980 - val_loss: 1.1094 - val_accuracy: 0.0944
Epoch 5/20
66/66 [==============================] - 9s 131ms/step - loss: 1.0752 - accuracy: 0.1082 - val_loss: 1.0852 - val_accuracy: 0.0962
Epoch 6/20
66/66 [==============================] - 6s 88ms/step - loss: 1.0429 - accuracy: 0.1192 - val_loss: 1.0085 - val_accuracy: 0.1409
Epoch 7/20
66/66 [==============================] - 6s 89ms/step - loss: 1.0094 - accuracy: 0.1305 - val_loss: 0.9751 - val_accuracy: 0.1475
Epoch 8/20

In [42]:
print("------------------------------LSTM MODEL OUTPUT------------------------------")
print(encoder1_loss, decoder1_loss)

------------------------------LSTM MODEL OUTPUT------------------------------
1.2310974597930908 0.06387358158826828


In [43]:
print("------------------------------RNN MODEL OUTPUT------------------------------")
print(encoder2_loss, decoder2_loss)

------------------------------RNN MODEL OUTPUT------------------------------
0.7825456857681274 0.199155792593956
